---
### Step 1: Load The Data

In [1]:
# Import libraries
import math
import cv2
import csv
import numpy as np
from PIL import Image 
import cvlib as cv
from cvlib.object_detection import draw_bbox
import matplotlib.pyplot as plt
#import tensorflow as tf
#from tensorflow.contrib.layers import flatten
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense, Flatten, Lambda, Conv2D, Activation, Dropout
from keras.callbacks import ModelCheckpoint
%matplotlib inline

Using TensorFlow backend.


In [2]:
lines = []
with open('labeled_images.csv') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)
        for line in reader:
            lines.append(line)
print(len(lines))

4110


In [3]:
#labeled_pictures = pd.read_csv('labeled_images.csv')
shuffle(lines)
train_data, val_data = train_test_split(lines, test_size = 0.2)
print(len(train_data))
print(len(val_data))

3288
822


In [4]:
train_images = []
train_labels = []
for line in train_data:
    img = cv2.cvtColor(cv2.imread('./rec_images/'+str(line[0])+'.jpeg'), cv2.COLOR_BGR2HSV)
    train_images.append(img)
    if int(line[1]) == 0: label = 1
    else: label = 0
    train_labels.append(label)
X_train = np.array(train_images)
y_train = np.array(train_labels)

In [5]:
val_images = []
val_labels = []
for line in val_data:
    img = cv2.cvtColor(cv2.imread('./rec_images/'+str(line[0])+'.jpeg'), cv2.COLOR_BGR2HSV)
    val_images.append(img)
    if int(line[1]) == 0: label = 1
    else: label = 0
    val_labels.append(label)
X_val = np.array(val_images)
y_val = np.array(val_labels)

In [ ]:
index = 4
test_img = X_train[index]
plt.imshow(test_img)
print(y_train[index])    
print(test_img.shape)

In [ ]:
def object_extract(image):   
    bbox, label, conf = cv.detect_common_objects(image)
    print(label)
    print(bbox)
    output_image = np.copy(image)
    draw_bbox(output_image, bbox, label, conf)
    plt.imshow(output_image)
    
    # Setting the points for cropped image 
    traffic_light_extraction = []
    for extraction in range(len(label)):
        if label[extraction] == 'traffic light':
            try:
                left = bbox[extraction][0]
                top = bbox[extraction][1]
                right = bbox[extraction][2]
                bottom = bbox[extraction][3]
                
                # Cropped image and resize dimension 
                # (It will not change orginal image) 
                crop_img = image[top:bottom, left:right]
                resized_image = cv2.resize(crop_img, (50, 100))
                traffic_light_extraction.append(resized_image)

            except IndexError:
                pass  
    return traffic_light_extraction

test = object_extract(test_img)

In [ ]:
#type(extraction)
#extraction.shape
plt.imshow(test[4])

---
### Step 2: Define Model

In [8]:
model = Sequential()
#normalize data
model.add(Lambda(lambda x: (x / 255.0) - 0.5,  input_shape=(600,800,3)))
#layer 1- Convolution, no of filters- 24, filter size= 5x5, stride= 2x2
model.add(Conv2D(24, (5, 5), strides=(2, 2), activation="elu"))
#layer 2- Convolution, no of filters- 36, filter size= 5x5, stride= 2x2
model.add(Conv2D(36, (5,5), strides=(2, 2), activation="elu"))
#layer 3- Convolution, no of filters- 48, filter size= 5x5, stride= 2x2
model.add(Conv2D(48, (5,5), strides=(2, 2), activation="elu"))
#layer 4- Convolution, no of filters- 64, filter size= 3x3, stride= 1x1
model.add(Conv2D(64, (3,3), activation="elu"))
#layer 5- Convolution, no of filters- 64, filter size= 3x3, stride= 1x1
model.add(Conv2D(64, (3,3), activation="elu"))
#flatten image from 2D to side by side
model.add(Flatten())
#layer 6- fully connected layer 1
model.add(Dense(100, activation="elu"))
#dropout layer to avoid overfitting
model.add(Dropout(0.25))
#layer 7- fully connected layer 1
model.add(Dense(50, activation="elu"))
#layer 8- fully connected layer 1
model.add(Dense(10, activation="elu"))
#layer 9- fully connected layer 1
model.add(Dense(1, activation="softmax"))

#model.compile(loss='categorical_crossentropy',optimizer='adam')
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
#model.fit_generator(generator = data_generator(train_data),
#                    validation_data = data_generator(val_data),
#                    epochs = 2,
#                    steps_per_epoch  = math.ceil(len(train_data) / 128),
#                    validation_steps = math.ceil(len(val_data)   / 128)    )
model.fit(X_train, y_train, epochs=5, batch_size=32)
score = model.evaluate(X_val, y_val, batch_size=32)

model.save('model.h5')
print('Model saved')
#model.summary()

Epoch 1/5
3288/3288 [==============================] - 774s 236ms/step - loss: 10.4004 - acc: 0.3476
Epoch 2/5
3232/3288 [============================>.] - ETA: 13s - loss: 10.4079 - acc: 0.3472

KeyboardInterrupt: 

In [7]:
score

[10.065812844139526, 0.3686131389761783]